In [1]:
import pandas as pd
import numpy as np
import sqlite3

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.collections import LineCollection
import matplotlib.dates as mdates
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from skopt import BayesSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from skopt.space import Real, Integer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import warnings
warnings.filterwarnings("ignore")


In [2]:
# Connect to the database
conn = sqlite3.connect('data/EPL_database.db')

# Example: list all tables
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
tables

,name
0,Raw_LongTermForm_Season2023_2024_Season2023_2024
1,Raw_ShortTermForm_Season2023_2024_Season2023_2024
2,Raw_LongTermForm_Season2017_2018
3,Raw_LongTermForm_Season2018_2019
4,Raw_LongTermForm_Season2019_2020
5,Raw_LongTermForm_Season2020_2021
6,Raw_LongTermForm_Season2021_2022
7,Raw_LongTermForm_Season2022_2023
8,Raw_LongTermForm_Season2023_2024
9,DataCoUk_Season2017_2018


In [3]:
# List of Databases we are going to use, which are all the matches between season 2017-18 to 2024-25
season_tables = [
    'DataCoUk_Season2017_2018',
    'DataCoUk_Season2018_2019',
    'DataCoUk_Season2019_2020',
    'DataCoUk_Season2020_2021',
    'DataCoUk_Season2021_2022',
    'DataCoUk_Season2022_2023',
    'DataCoUk_Season2023_2024',
    'DataCoUk_Season2024_2025'
]

# Merge all into a single DataFrame
db = pd.concat(
    [pd.read_sql_query(f"SELECT * FROM {table}", conn) for table in season_tables],
    ignore_index=True
)

# Optional: preview the result
db.head()


,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,1XBCH,1XBCD,1XBCA,BFECH,BFECD,BFECA,BFEC>2.5,BFEC<2.5,BFECAHH,BFECAHA
0,E0,11/08/2017,Arsenal,Leicester,4,3,H,2,2,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,12/08/2017,Brighton,Manchester City,0,2,A,0,0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,12/08/2017,Chelsea,Burnley,2,3,A,0,3,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,12/08/2017,Crystal Palace,Huddersfield,0,3,A,0,2,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E0,12/08/2017,Everton,Stoke,1,0,H,1,0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
db.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2690 entries, 0 to 2689
Columns: 153 entries, Div to BFECAHA
dtypes: float64(129), int64(16), object(8)
memory usage: 3.1+ MB


In [5]:
len(db)

2690

In [6]:
print(db.columns)


Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR',
       ...
       '1XBCH', '1XBCD', '1XBCA', 'BFECH', 'BFECD', 'BFECA', 'BFEC>2.5',
       'BFEC<2.5', 'BFECAHH', 'BFECAHA'],
      dtype='object', length=153)


In [7]:
pd.set_option('display.max_columns', None)
print(db.columns.tolist())


['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'Bb1X2', 'BbMxH', 'BbAvH', 'BbMxD', 'BbAvD', 'BbMxA', 'BbAvA', 'BbOU', 'BbMx>2.5', 'BbAv>2.5', 'BbMx<2.5', 'BbAv<2.5', 'BbAH', 'BbAHh', 'BbMxAHH', 'BbAvAHH', 'BbMxAHA', 'BbAvAHA', 'PSCH', 'PSCD', 'PSCA', 'Time', 'MaxH', 'MaxD', 'MaxA', 'AvgH', 'AvgD', 'AvgA', 'B365>2.5', 'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5', 'AHh', 'B365AHH', 'B365AHA', 'PAHH', 'PAHA', 'MaxAHH', 'MaxAHA', 'AvgAHH', 'AvgAHA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'BWCA', 'IWCH', 'IWCD', 'IWCA', 'WHCH', 'WHCD', 'WHCA', 'VCCH', 'VCCD', 'VCCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD', 'AvgCA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5', 'MaxC>2.5', 'MaxC<2

We noticed that in some years the bookmakers are some, while on other years there are others. Furthermore, we don't really need all the different bookmakers predicting home winning, draw or away winning, so what we can do is create 3 single colums, indentical throughout all years, that are called odds_hw (odds home winning), odds_d (odds draw), odds_aw (odds away winning).

In [8]:
# Define groups of columns for home win, draw, and away win odds
home_win_cols = ['B365H', 'BWH', 'IWH', 'LBH', 'PSH', 'WHH', 'VCH', 'MaxH', 'AvgH', 'PSCH', 'B365CH', 'BWCH', 'IWCH', 'WHCH', 'VCCH', 'MaxCH', 'AvgCH', 'BFH', 'BFCH', '1XBH', '1XBCH']
draw_cols     = ['B365D', 'BWD', 'IWD', 'LBD', 'PSD', 'WHD', 'VCD', 'MaxD', 'AvgD', 'PSCD', 'B365CD', 'BWCD', 'IWCD', 'WHCD', 'VCCD', 'MaxCD', 'AvgCD', 'BFD', 'BFCD', '1XBD', '1XBCD']
away_win_cols = ['B365A', 'BWA', 'IWA', 'LBA', 'PSA', 'WHA', 'VCA', 'MaxA', 'AvgA', 'PSCA', 'B365CA', 'BWCA', 'IWCA', 'WHCA', 'VCCA', 'MaxCA', 'AvgCA', 'BFA', 'BFCA', '1XBA', '1XBCA']

# Compute row-wise means (ignoring NaNs)
db['odds_hw'] = db[home_win_cols].mean(axis=1)
db['odds_d']  = db[draw_cols].mean(axis=1)
db['odds_aw'] = db[away_win_cols].mean(axis=1)

# Drop the original odds columns
db.drop(columns=home_win_cols + draw_cols + away_win_cols, inplace=True)

In [9]:
# Checking if it worked
db[['HomeTeam', 'AwayTeam', 'odds_hw', 'odds_d', 'odds_aw']].head()

,HomeTeam,AwayTeam,odds_hw,odds_d,odds_aw
0,Arsenal,Leicester,1.50250,4.49750,6.60625
1,Brighton,Manchester City,10.33750,5.38125,1.32125
2,Chelsea,Burnley,1.25375,6.05000,13.93750
3,Crystal Palace,Huddersfield,1.81625,3.50500,4.90875
4,Everton,Stoke,1.71125,3.66500,5.49750


We are now going to plot some graphs to see interesting patterns....

Since we are going to keep the temporal order, we are dropping the 'Date' columns and also the 'Div' one because is irrelevant

In [10]:
dropping_columns = ['Div', 'Date']

db.drop(columns=dropping_columns, inplace=True)

We are going to do now the same with all ods that say over 2.5 and under 2.5

In [11]:
# Define columns for over and under 2.5 goals
over_cols = [
    'BbMx>2.5', 'BbAv>2.5', 'B365>2.5', 'P>2.5', 'Max>2.5', 'Avg>2.5',
    'B365C>2.5', 'PC>2.5', 'MaxC>2.5', 'AvgC>2.5', 'BFE>2.5', 'BFEC>2.5'
]

under_cols = [
    'BbMx<2.5', 'BbAv<2.5', 'B365<2.5', 'P<2.5', 'Max<2.5', 'Avg<2.5',
    'B365C<2.5', 'PC<2.5', 'MaxC<2.5', 'AvgC<2.5', 'BFE<2.5', 'BFEC<2.5'
]

# Create new averaged columns
db['odds_over2_5'] = db[over_cols].mean(axis=1)
db['odds_under2_5'] = db[under_cols].mean(axis=1)

db.drop(columns=over_cols + under_cols, inplace=True)


In [12]:
# Checking if it worked
db[['HomeTeam', 'AwayTeam', 'odds_over2_5', 'odds_under2_5']].head()

,HomeTeam,AwayTeam,odds_over2_5,odds_under2_5
0,Arsenal,Leicester,1.630,2.375
1,Brighton,Manchester City,1.665,2.335
2,Chelsea,Burnley,1.685,2.280
3,Crystal Palace,Huddersfield,2.150,1.755
4,Everton,Stoke,2.125,1.780


In [13]:
pd.set_option('display.max_columns', None)
print(db.columns.tolist())

['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'Bb1X2', 'BbMxH', 'BbAvH', 'BbMxD', 'BbAvD', 'BbMxA', 'BbAvA', 'BbOU', 'BbAH', 'BbAHh', 'BbMxAHH', 'BbAvAHH', 'BbMxAHA', 'BbAvAHA', 'Time', 'AHh', 'B365AHH', 'B365AHA', 'PAHH', 'PAHA', 'MaxAHH', 'MaxAHA', 'AvgAHH', 'AvgAHA', 'AHCh', 'B365CAHH', 'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA', 'AvgCAHH', 'AvgCAHA', 'BFEH', 'BFED', 'BFEA', 'BFEAHH', 'BFEAHA', 'BFECH', 'BFECD', 'BFECA', 'BFECAHH', 'BFECAHA', 'odds_hw', 'odds_d', 'odds_aw', 'odds_over2_5', 'odds_under2_5']


We decided now, after analyzing the whole dataset, to drop a lot of the existing columns because they show either:
- Max odds a bookmaker has for a certain game, which is irrelevant
- Minimum odds a bookmaker has for a certain game, which is also irrelevant
- Average of the odds, which we alredy have calcualted (we didn't know this column was there)
- Half time data: since we want to predict the output of a game before it starts, we can't have halftime data otherwise it will introduce leakage
- Exchange odds, which again we don't need

In [14]:
columns_to_drop = [
    'Bb1X2', 'BbMxH', 'BbAvH', 'BbMxD', 'BbAvD', 'BbMxA', 'BbAvA',
    'BbOU', 'BbAH', 'BbAHh', 'BbMxAHH', 'BbAvAHH', 'BbMxAHA', 'BbAvAHA',
    'AHh', 'MaxAHH', 'MaxAHA', 'AvgAHH', 'AvgAHA',
    'MaxCAHH', 'MaxCAHA', 'AvgCAHH', 'AvgCAHA',
    'BFEH', 'BFED', 'BFEA', 'BFECH', 'BFECD', 'BFECA', 'HTHG', 'HTAG', 'HTR', 'AHCh'
]

db.drop(columns=columns_to_drop, inplace=True)


And now we will merge and find the mean of all the asian odds for both home and away, as we did before

In [15]:
asian_home_cols = ['B365AHH', 'PAHH', 'B365CAHH', 'PCAHH', 'BFEAHH', 'BFECAHH']
db['odds_home_asian'] = db[asian_home_cols].mean(axis=1)

asian_away_cols = ['B365AHA', 'PAHA', 'B365CAHA', 'PCAHA', 'BFEAHA', 'BFECAHA']
db['odds_away_asian'] = db[asian_away_cols].mean(axis=1)

cols_to_drop_asian = [
    'B365AHH', 'PAHH', 'B365CAHH', 'PCAHH', 'BFEAHH', 'BFECAHH',
    'B365AHA', 'PAHA', 'B365CAHA', 'PCAHA', 'BFEAHA', 'BFECAHA'
]

db.drop(columns=cols_to_drop_asian, inplace=True)



In [16]:
# Checking if it worked
db[['HomeTeam', 'AwayTeam', 'odds_home_asian', 'odds_away_asian']].head()

,HomeTeam,AwayTeam,odds_home_asian,odds_away_asian
0,Arsenal,Leicester,NaN,NaN
1,Brighton,Manchester City,NaN,NaN
2,Chelsea,Burnley,NaN,NaN
3,Crystal Palace,Huddersfield,NaN,NaN
4,Everton,Stoke,NaN,NaN


In [17]:
# Checking if it worked
db[['HomeTeam', 'AwayTeam', 'odds_home_asian', 'odds_away_asian']].tail()

,HomeTeam,AwayTeam,odds_home_asian,odds_away_asian
2685,Nottingham Forest,Wolverhampton Wanderers,2.036667,1.905000
2686,West Ham,Manchester City,1.936667,1.991667
2687,Chelsea,Crystal Palace,1.950000,1.985000
2688,Newcastle United,Tottenham,1.928333,2.008333
2689,Manchester United,Liverpool,2.005000,1.933333


We can conclude that not all the data has asian odds, but that is not a problem

Let's look at the new dataset now:

In [18]:
db.head()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,Time,odds_hw,odds_d,odds_aw,odds_over2_5,odds_under2_5,odds_home_asian,odds_away_asian
0,Arsenal,Leicester,4,3,H,M Dean,27,6,10,3,9,12,9,4,0,1,0,0,NaN,1.50250,4.49750,6.60625,1.630,2.375,NaN,NaN
1,Brighton,Manchester City,0,2,A,M Oliver,6,14,2,4,6,9,3,10,0,2,0,0,NaN,10.33750,5.38125,1.32125,1.665,2.335,NaN,NaN
2,Chelsea,Burnley,2,3,A,C Pawson,19,10,6,5,16,11,8,5,3,3,2,0,NaN,1.25375,6.05000,13.93750,1.685,2.280,NaN,NaN
3,Crystal Palace,Huddersfield,0,3,A,J Moss,14,8,4,6,7,19,12,9,1,3,0,0,NaN,1.81625,3.50500,4.90875,2.150,1.755,NaN,NaN
4,Everton,Stoke,1,0,H,N Swarbrick,9,9,4,1,13,10,6,7,1,1,0,0,NaN,1.71125,3.66500,5.49750,2.125,1.780,NaN,NaN


Way more clean and understandable, now we can proceede with plotting the graphs and see how our database reacts

To better understand what we have now, the columns with odd names are:
- FTHG and FTAG (our target variables): Full Time Home Goals and Full Time Away Goals
- FTR: Full Time Result
- HS and AS: Home Shots and Away Shots
- HST and AST: Home Shots on Target and Away Shots on Target
- HF and AF: Home Fouls and Away Fouls
- HC and AC: Home Corners and Away Corners
- HY and AY: Home Yellows (cards) and Away Yellows
- HR and AR: Home Reds (cards) and Away Reds

However, after further thought, we came with the understanding that features such as Shots, Shots on target, Fouls commited, Cards... That is, every in-game stat, cannot be used since we are trying to predict the result of a match before that match starts, so in a real world scenario we wouldn't have those information.  
However this doesn't mean that we have to drop those columns and that's it. We can still extract some valuable information from those columns, such as:  
- 'avg_goals_L10_home' and 'avg_goals_L10_away': The average goals scored by the home team in their last 10 home matches and the average goals scored by the away team in their last 10 away matches
- 'avg_shots_home_L10' and 'avg_shots_away_L10': The average shots by the home team in their last 10 home matches and the average shots by the away team in their last 10 away matches
- 'avg_shots_ont_home_L10' and 'avg_shots_ont_away_L10': The average shots on target by the home team in their last 10 home matches and the average shots on target by the away team in their last away 10 matches
- 'avg_corners_home_L10' and 'avg_corners_away_L10': The average corners taken by the home team in their last 10 home matches and the average corners taken by the away team in their last 10 away matches
- 'avg_goals_conceded_home_L10' and 'avg_goals_conceded_away_L10': Average goals conceeded by the home team in their last 10 home matches and average goals conceeded by the away team in their last 10 away matches
- 'avg_fouls_committed_home_L10' and 'avg_fouls_committed_away_L10': Average fouls committed by the home team in their last 10 home matches and average fouls committed by the away team in their last 10 away matches
- 'win_rate_home_L10' and 'win_rate_away_L10': Percentage of wins by the home team in their last 10 home games and percentage of wins by the away team in their last 10 away games
- 'draw_rate_home_L10' and 'draw_rate_away_L10': Same as above but for draws
- 'loss_rate_home_L10' and 'loss_rate_away_L10': Same as above but for losses
- 'home_strength_diff': The difference in net goal strength between home and away teams over their last 10 matches, where net strength = average goals scored minus average goals conceded, meaning that if the value in negative, the away team is stronger, and if is positive, the home team is stronger
- 'offensive_index_home' and 'offensive_index_away': Weighted combination of goals scored, shots, and shots on target in the last 10 matches for home and away teams, measuring their attacking ability.
- 'defensive_solidity_home' and 'defensive_solidity_away': Weighted combination of goals conceded, fouls committed, and inverse of win rate in the last 10 matches for home and away teams, measuring their defensive weakness (higher value = worse defense).
- 'avg_home_yellow_L10' and 'avg_away_yellow_L10': The average number of yellow cards received by the home and away teams respectively in their last 10 matches.
- 'avg_home_red_L10' and 'avg_away_red_L10': The average number of red cards received by the home and away teams respectively in their last 10 matches.
- 'h2h_avg_home_goals_last_5': The average number of goals the home team scored in its last 5 games against the current away team (home or away).
- 'h2h_avg_away_goals_last_5': The same but for the away team.

In [19]:
# Reset index to ensure consistent iteration
db.reset_index(drop=True, inplace=True)

# Initialize new columns
db['avg_goals_L10_home'] = None
db['avg_goals_L10_away'] = None

# Loop to calculate rolling averages
home_goals = []
away_goals = []

for idx, row in db.iterrows():
    home_team = row['HomeTeam']
    away_team = row['AwayTeam']

    # Get last 10 home games for this home team before current match
    past_home = db[(db.index < idx) & (db['HomeTeam'] == home_team)]
    avg_home_goals = past_home['FTHG'].tail(10).mean()
    home_goals.append(avg_home_goals)

    # Get last 10 away games for this away team before current match
    past_away = db[(db.index < idx) & (db['AwayTeam'] == away_team)]
    avg_away_goals = past_away['FTAG'].tail(10).mean()
    away_goals.append(avg_away_goals)

# Assign computed values
db['avg_goals_L10_home'] = home_goals
db['avg_goals_L10_away'] = away_goals


In [20]:
db.tail()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,Time,odds_hw,odds_d,odds_aw,odds_over2_5,odds_under2_5,odds_home_asian,odds_away_asian,avg_goals_L10_home,avg_goals_L10_away
2685,Nottingham Forest,Wolverhampton Wanderers,1,1,D,S Hooper,16,11,5,3,15,18,7,3,3,4,0,0,15:00,2.024375,3.643125,3.720000,1.879,2.030,2.036667,1.905000,1.4,1.0
2686,West Ham,Manchester City,1,3,A,M Oliver,10,23,2,8,10,3,3,11,3,2,0,0,17:30,7.575000,5.280625,1.396875,1.518,2.690,1.936667,1.991667,1.5,2.6
2687,Chelsea,Crystal Palace,1,1,D,J Gillett,13,9,7,3,9,13,4,5,4,2,0,0,13:30,1.643125,4.344375,5.000000,1.545,2.599,1.950000,1.985000,2.7,1.0
2688,Newcastle United,Tottenham,2,1,H,R Jones,9,20,3,6,16,13,7,12,4,4,0,0,13:30,2.500000,3.950000,2.596250,1.405,3.118,1.928333,2.008333,2.7,1.5
2689,Manchester United,Liverpool,0,3,A,A Taylor,8,11,3,3,7,7,5,2,4,1,0,0,16:00,3.720000,4.125625,1.901875,1.462,2.871,2.005000,1.933333,1.9,2.2


In [21]:
# Initialize new columns
db['avg_shots_home_L10'] = None
db['avg_shots_away_L10'] = None

# Loop to calculate rolling average shots
shots_home = []
shots_away = []

for idx, row in db.iterrows():
    home_team = row['HomeTeam']
    away_team = row['AwayTeam']

    # Last 10 home games for home team
    past_home = db[(db.index < idx) & (db['HomeTeam'] == home_team)]
    avg_home_shots = past_home['HS'].tail(10).mean()
    shots_home.append(avg_home_shots)

    # Last 10 away games for away team
    past_away = db[(db.index < idx) & (db['AwayTeam'] == away_team)]
    avg_away_shots = past_away['AS'].tail(10).mean()
    shots_away.append(avg_away_shots)

# Assign values
db['avg_shots_home_L10'] = shots_home
db['avg_shots_away_L10'] = shots_away


In [22]:
db.tail()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,Time,odds_hw,odds_d,odds_aw,odds_over2_5,odds_under2_5,odds_home_asian,odds_away_asian,avg_goals_L10_home,avg_goals_L10_away,avg_shots_home_L10,avg_shots_away_L10
2685,Nottingham Forest,Wolverhampton Wanderers,1,1,D,S Hooper,16,11,5,3,15,18,7,3,3,4,0,0,15:00,2.024375,3.643125,3.720000,1.879,2.030,2.036667,1.905000,1.4,1.0,14.3,9.3
2686,West Ham,Manchester City,1,3,A,M Oliver,10,23,2,8,10,3,3,11,3,2,0,0,17:30,7.575000,5.280625,1.396875,1.518,2.690,1.936667,1.991667,1.5,2.6,14.1,15.5
2687,Chelsea,Crystal Palace,1,1,D,J Gillett,13,9,7,3,9,13,4,5,4,2,0,0,13:30,1.643125,4.344375,5.000000,1.545,2.599,1.950000,1.985000,2.7,1.0,18.6,9.5
2688,Newcastle United,Tottenham,2,1,H,R Jones,9,20,3,6,16,13,7,12,4,4,0,0,13:30,2.500000,3.950000,2.596250,1.405,3.118,1.928333,2.008333,2.7,1.5,16.3,13.5
2689,Manchester United,Liverpool,0,3,A,A Taylor,8,11,3,3,7,7,5,2,4,1,0,0,16:00,3.720000,4.125625,1.901875,1.462,2.871,2.005000,1.933333,1.9,2.2,16.3,18.6


In [23]:
# Initialize new columns
db['avg_shots_ont_home_L10'] = None
db['avg_shots_ont_away_L10'] = None

# Loop to calculate rolling average shots on target
shots_ont_home = []
shots_ont_away = []

for idx, row in db.iterrows():
    home_team = row['HomeTeam']
    away_team = row['AwayTeam']

    # Last 10 home games for home team
    past_home = db[(db.index < idx) & (db['HomeTeam'] == home_team)]
    avg_home_shots_ont = past_home['HST'].tail(10).mean()
    shots_ont_home.append(avg_home_shots_ont)

    # Last 10 away games for away team
    past_away = db[(db.index < idx) & (db['AwayTeam'] == away_team)]
    avg_away_shots_ont = past_away['AST'].tail(10).mean()
    shots_ont_away.append(avg_away_shots_ont)

# Assign values
db['avg_shots_ont_home_L10'] = shots_ont_home
db['avg_shots_ont_away_L10'] = shots_ont_away


In [24]:
db.tail()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,Time,odds_hw,odds_d,odds_aw,odds_over2_5,odds_under2_5,odds_home_asian,odds_away_asian,avg_goals_L10_home,avg_goals_L10_away,avg_shots_home_L10,avg_shots_away_L10,avg_shots_ont_home_L10,avg_shots_ont_away_L10
2685,Nottingham Forest,Wolverhampton Wanderers,1,1,D,S Hooper,16,11,5,3,15,18,7,3,3,4,0,0,15:00,2.024375,3.643125,3.720000,1.879,2.030,2.036667,1.905000,1.4,1.0,14.3,9.3,4.8,3.6
2686,West Ham,Manchester City,1,3,A,M Oliver,10,23,2,8,10,3,3,11,3,2,0,0,17:30,7.575000,5.280625,1.396875,1.518,2.690,1.936667,1.991667,1.5,2.6,14.1,15.5,4.6,7.6
2687,Chelsea,Crystal Palace,1,1,D,J Gillett,13,9,7,3,9,13,4,5,4,2,0,0,13:30,1.643125,4.344375,5.000000,1.545,2.599,1.950000,1.985000,2.7,1.0,18.6,9.5,7.8,3.8
2688,Newcastle United,Tottenham,2,1,H,R Jones,9,20,3,6,16,13,7,12,4,4,0,0,13:30,2.500000,3.950000,2.596250,1.405,3.118,1.928333,2.008333,2.7,1.5,16.3,13.5,5.8,5.3
2689,Manchester United,Liverpool,0,3,A,A Taylor,8,11,3,3,7,7,5,2,4,1,0,0,16:00,3.720000,4.125625,1.901875,1.462,2.871,2.005000,1.933333,1.9,2.2,16.3,18.6,6.7,5.9


In [25]:
# Initialize new columns
db['avg_corners_home_L10'] = None
db['avg_corners_away_L10'] = None

# Loop to calculate rolling average corners
corners_home = []
corners_away = []

for idx, row in db.iterrows():
    home_team = row['HomeTeam']
    away_team = row['AwayTeam']

    # Last 10 home games for home team
    past_home = db[(db.index < idx) & (db['HomeTeam'] == home_team)]
    avg_home_corners = past_home['HC'].tail(10).mean()
    corners_home.append(avg_home_corners)

    # Last 10 away games for away team
    past_away = db[(db.index < idx) & (db['AwayTeam'] == away_team)]
    avg_away_corners = past_away['AC'].tail(10).mean()
    corners_away.append(avg_away_corners)

# Assign values
db['avg_corners_home_L10'] = corners_home
db['avg_corners_away_L10'] = corners_away


In [26]:
db.tail()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,Time,odds_hw,odds_d,odds_aw,odds_over2_5,odds_under2_5,odds_home_asian,odds_away_asian,avg_goals_L10_home,avg_goals_L10_away,avg_shots_home_L10,avg_shots_away_L10,avg_shots_ont_home_L10,avg_shots_ont_away_L10,avg_corners_home_L10,avg_corners_away_L10
2685,Nottingham Forest,Wolverhampton Wanderers,1,1,D,S Hooper,16,11,5,3,15,18,7,3,3,4,0,0,15:00,2.024375,3.643125,3.720000,1.879,2.030,2.036667,1.905000,1.4,1.0,14.3,9.3,4.8,3.6,4.2,2.5
2686,West Ham,Manchester City,1,3,A,M Oliver,10,23,2,8,10,3,3,11,3,2,0,0,17:30,7.575000,5.280625,1.396875,1.518,2.690,1.936667,1.991667,1.5,2.6,14.1,15.5,4.6,7.6,4.9,7.0
2687,Chelsea,Crystal Palace,1,1,D,J Gillett,13,9,7,3,9,13,4,5,4,2,0,0,13:30,1.643125,4.344375,5.000000,1.545,2.599,1.950000,1.985000,2.7,1.0,18.6,9.5,7.8,3.8,6.6,4.4
2688,Newcastle United,Tottenham,2,1,H,R Jones,9,20,3,6,16,13,7,12,4,4,0,0,13:30,2.500000,3.950000,2.596250,1.405,3.118,1.928333,2.008333,2.7,1.5,16.3,13.5,5.8,5.3,5.9,6.6
2689,Manchester United,Liverpool,0,3,A,A Taylor,8,11,3,3,7,7,5,2,4,1,0,0,16:00,3.720000,4.125625,1.901875,1.462,2.871,2.005000,1.933333,1.9,2.2,16.3,18.6,6.7,5.9,6.9,7.7


In [27]:
# Initialize new columns
db['avg_goals_conceded_home_L10'] = None
db['avg_goals_conceded_away_L10'] = None

# Loop to calculate rolling average goals conceded
gc_home = []
gc_away = []

for idx, row in db.iterrows():
    home_team = row['HomeTeam']
    away_team = row['AwayTeam']

    # Last 10 home games of the home team
    past_home = db[(db.index < idx) & (db['HomeTeam'] == home_team)]
    avg_gc_home = past_home['FTAG'].tail(10).mean()
    gc_home.append(avg_gc_home)

    # Last 10 away games of the away team
    past_away = db[(db.index < idx) & (db['AwayTeam'] == away_team)]
    avg_gc_away = past_away['FTHG'].tail(10).mean()
    gc_away.append(avg_gc_away)

# Assign values
db['avg_goals_conceded_home_L10'] = gc_home
db['avg_goals_conceded_away_L10'] = gc_away


In [28]:
db.tail()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,Time,odds_hw,odds_d,odds_aw,odds_over2_5,odds_under2_5,odds_home_asian,odds_away_asian,avg_goals_L10_home,avg_goals_L10_away,avg_shots_home_L10,avg_shots_away_L10,avg_shots_ont_home_L10,avg_shots_ont_away_L10,avg_corners_home_L10,avg_corners_away_L10,avg_goals_conceded_home_L10,avg_goals_conceded_away_L10
2685,Nottingham Forest,Wolverhampton Wanderers,1,1,D,S Hooper,16,11,5,3,15,18,7,3,3,4,0,0,15:00,2.024375,3.643125,3.720000,1.879,2.030,2.036667,1.905000,1.4,1.0,14.3,9.3,4.8,3.6,4.2,2.5,1.6,2.0
2686,West Ham,Manchester City,1,3,A,M Oliver,10,23,2,8,10,3,3,11,3,2,0,0,17:30,7.575000,5.280625,1.396875,1.518,2.690,1.936667,1.991667,1.5,2.6,14.1,15.5,4.6,7.6,4.9,7.0,2.0,0.6
2687,Chelsea,Crystal Palace,1,1,D,J Gillett,13,9,7,3,9,13,4,5,4,2,0,0,13:30,1.643125,4.344375,5.000000,1.545,2.599,1.950000,1.985000,2.7,1.0,18.6,9.5,7.8,3.8,6.6,4.4,1.4,1.9
2688,Newcastle United,Tottenham,2,1,H,R Jones,9,20,3,6,16,13,7,12,4,4,0,0,13:30,2.500000,3.950000,2.596250,1.405,3.118,1.928333,2.008333,2.7,1.5,16.3,13.5,5.8,5.3,5.9,6.6,1.5,1.9
2689,Manchester United,Liverpool,0,3,A,A Taylor,8,11,3,3,7,7,5,2,4,1,0,0,16:00,3.720000,4.125625,1.901875,1.462,2.871,2.005000,1.933333,1.9,2.2,16.3,18.6,6.7,5.9,6.9,7.7,1.2,1.4


In [29]:
# Initialize new columns
db['avg_fouls_committed_home_L10'] = None
db['avg_fouls_committed_away_L10'] = None

# Loop to calculate rolling average fouls committed
fouls_home = []
fouls_away = []

for idx, row in db.iterrows():
    home_team = row['HomeTeam']
    away_team = row['AwayTeam']

    # Last 10 home games of home team
    past_home = db[(db.index < idx) & (db['HomeTeam'] == home_team)]
    avg_fouls_home = past_home['HF'].tail(10).mean()
    fouls_home.append(avg_fouls_home)

    # Last 10 away games of away team
    past_away = db[(db.index < idx) & (db['AwayTeam'] == away_team)]
    avg_fouls_away = past_away['AF'].tail(10).mean()
    fouls_away.append(avg_fouls_away)

# Assign values
db['avg_fouls_committed_home_L10'] = fouls_home
db['avg_fouls_committed_away_L10'] = fouls_away


In [30]:
db.tail()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,Time,odds_hw,odds_d,odds_aw,odds_over2_5,odds_under2_5,odds_home_asian,odds_away_asian,avg_goals_L10_home,avg_goals_L10_away,avg_shots_home_L10,avg_shots_away_L10,avg_shots_ont_home_L10,avg_shots_ont_away_L10,avg_corners_home_L10,avg_corners_away_L10,avg_goals_conceded_home_L10,avg_goals_conceded_away_L10,avg_fouls_committed_home_L10,avg_fouls_committed_away_L10
2685,Nottingham Forest,Wolverhampton Wanderers,1,1,D,S Hooper,16,11,5,3,15,18,7,3,3,4,0,0,15:00,2.024375,3.643125,3.720000,1.879,2.030,2.036667,1.905000,1.4,1.0,14.3,9.3,4.8,3.6,4.2,2.5,1.6,2.0,11.4,13.1
2686,West Ham,Manchester City,1,3,A,M Oliver,10,23,2,8,10,3,3,11,3,2,0,0,17:30,7.575000,5.280625,1.396875,1.518,2.690,1.936667,1.991667,1.5,2.6,14.1,15.5,4.6,7.6,4.9,7.0,2.0,0.6,11.3,6.5
2687,Chelsea,Crystal Palace,1,1,D,J Gillett,13,9,7,3,9,13,4,5,4,2,0,0,13:30,1.643125,4.344375,5.000000,1.545,2.599,1.950000,1.985000,2.7,1.0,18.6,9.5,7.8,3.8,6.6,4.4,1.4,1.9,10.5,13.2
2688,Newcastle United,Tottenham,2,1,H,R Jones,9,20,3,6,16,13,7,12,4,4,0,0,13:30,2.500000,3.950000,2.596250,1.405,3.118,1.928333,2.008333,2.7,1.5,16.3,13.5,5.8,5.3,5.9,6.6,1.5,1.9,10.2,11.4
2689,Manchester United,Liverpool,0,3,A,A Taylor,8,11,3,3,7,7,5,2,4,1,0,0,16:00,3.720000,4.125625,1.901875,1.462,2.871,2.005000,1.933333,1.9,2.2,16.3,18.6,6.7,5.9,6.9,7.7,1.2,1.4,9.3,13.0


In [31]:
# Initialize new columns
db['win_rate_home_L10'] = None
db['win_rate_away_L10'] = None

# Loop to calculate win rates
win_rate_home = []
win_rate_away = []

for idx, row in db.iterrows():
    home_team = row['HomeTeam']
    away_team = row['AwayTeam']

    # Last 10 home games of home team
    past_home = db[(db.index < idx) & (db['HomeTeam'] == home_team)]
    recent_home = past_home.tail(10)
    home_wins = (recent_home['FTR'] == 'H').sum()
    win_rate_home.append(home_wins / len(recent_home) if len(recent_home) > 0 else None)

    # Last 10 away games of away team
    past_away = db[(db.index < idx) & (db['AwayTeam'] == away_team)]
    recent_away = past_away.tail(10)
    away_wins = (recent_away['FTR'] == 'A').sum()
    win_rate_away.append(away_wins / len(recent_away) if len(recent_away) > 0 else None)

# Assign values
db['win_rate_home_L10'] = win_rate_home
db['win_rate_away_L10'] = win_rate_away


In [32]:
db.tail()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,Time,odds_hw,odds_d,odds_aw,odds_over2_5,odds_under2_5,odds_home_asian,odds_away_asian,avg_goals_L10_home,avg_goals_L10_away,avg_shots_home_L10,avg_shots_away_L10,avg_shots_ont_home_L10,avg_shots_ont_away_L10,avg_corners_home_L10,avg_corners_away_L10,avg_goals_conceded_home_L10,avg_goals_conceded_away_L10,avg_fouls_committed_home_L10,avg_fouls_committed_away_L10,win_rate_home_L10,win_rate_away_L10
2685,Nottingham Forest,Wolverhampton Wanderers,1,1,D,S Hooper,16,11,5,3,15,18,7,3,3,4,0,0,15:00,2.024375,3.643125,3.720000,1.879,2.030,2.036667,1.905000,1.4,1.0,14.3,9.3,4.8,3.6,4.2,2.5,1.6,2.0,11.4,13.1,0.2,0.2
2686,West Ham,Manchester City,1,3,A,M Oliver,10,23,2,8,10,3,3,11,3,2,0,0,17:30,7.575000,5.280625,1.396875,1.518,2.690,1.936667,1.991667,1.5,2.6,14.1,15.5,4.6,7.6,4.9,7.0,2.0,0.6,11.3,6.5,0.2,0.9
2687,Chelsea,Crystal Palace,1,1,D,J Gillett,13,9,7,3,9,13,4,5,4,2,0,0,13:30,1.643125,4.344375,5.000000,1.545,2.599,1.950000,1.985000,2.7,1.0,18.6,9.5,7.8,3.8,6.6,4.4,1.4,1.9,10.5,13.2,0.7,0.2
2688,Newcastle United,Tottenham,2,1,H,R Jones,9,20,3,6,16,13,7,12,4,4,0,0,13:30,2.500000,3.950000,2.596250,1.405,3.118,1.928333,2.008333,2.7,1.5,16.3,13.5,5.8,5.3,5.9,6.6,1.5,1.9,10.2,11.4,0.5,0.2
2689,Manchester United,Liverpool,0,3,A,A Taylor,8,11,3,3,7,7,5,2,4,1,0,0,16:00,3.720000,4.125625,1.901875,1.462,2.871,2.005000,1.933333,1.9,2.2,16.3,18.6,6.7,5.9,6.9,7.7,1.2,1.4,9.3,13.0,0.5,0.5


In [33]:
# Initialize new columns
db['draw_rate_home_L10'] = None
db['draw_rate_away_L10'] = None

# Loop to calculate draw rates
draw_rate_home = []
draw_rate_away = []

for idx, row in db.iterrows():
    home_team = row['HomeTeam']
    away_team = row['AwayTeam']

    # Last 10 home games of home team
    past_home = db[(db.index < idx) & (db['HomeTeam'] == home_team)]
    recent_home = past_home.tail(10)
    home_draws = (recent_home['FTR'] == 'D').sum()
    draw_rate_home.append(home_draws / len(recent_home) if len(recent_home) > 0 else None)

    # Last 10 away games of away team
    past_away = db[(db.index < idx) & (db['AwayTeam'] == away_team)]
    recent_away = past_away.tail(10)
    away_draws = (recent_away['FTR'] == 'D').sum()
    draw_rate_away.append(away_draws / len(recent_away) if len(recent_away) > 0 else None)

# Assign values
db['draw_rate_home_L10'] = draw_rate_home
db['draw_rate_away_L10'] = draw_rate_away


In [34]:
db.tail()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,Time,odds_hw,odds_d,odds_aw,odds_over2_5,odds_under2_5,odds_home_asian,odds_away_asian,avg_goals_L10_home,avg_goals_L10_away,avg_shots_home_L10,avg_shots_away_L10,avg_shots_ont_home_L10,avg_shots_ont_away_L10,avg_corners_home_L10,avg_corners_away_L10,avg_goals_conceded_home_L10,avg_goals_conceded_away_L10,avg_fouls_committed_home_L10,avg_fouls_committed_away_L10,win_rate_home_L10,win_rate_away_L10,draw_rate_home_L10,draw_rate_away_L10
2685,Nottingham Forest,Wolverhampton Wanderers,1,1,D,S Hooper,16,11,5,3,15,18,7,3,3,4,0,0,15:00,2.024375,3.643125,3.720000,1.879,2.030,2.036667,1.905000,1.4,1.0,14.3,9.3,4.8,3.6,4.2,2.5,1.6,2.0,11.4,13.1,0.2,0.2,0.3,0.3
2686,West Ham,Manchester City,1,3,A,M Oliver,10,23,2,8,10,3,3,11,3,2,0,0,17:30,7.575000,5.280625,1.396875,1.518,2.690,1.936667,1.991667,1.5,2.6,14.1,15.5,4.6,7.6,4.9,7.0,2.0,0.6,11.3,6.5,0.2,0.9,0.5,0.1
2687,Chelsea,Crystal Palace,1,1,D,J Gillett,13,9,7,3,9,13,4,5,4,2,0,0,13:30,1.643125,4.344375,5.000000,1.545,2.599,1.950000,1.985000,2.7,1.0,18.6,9.5,7.8,3.8,6.6,4.4,1.4,1.9,10.5,13.2,0.7,0.2,0.1,0.3
2688,Newcastle United,Tottenham,2,1,H,R Jones,9,20,3,6,16,13,7,12,4,4,0,0,13:30,2.500000,3.950000,2.596250,1.405,3.118,1.928333,2.008333,2.7,1.5,16.3,13.5,5.8,5.3,5.9,6.6,1.5,1.9,10.2,11.4,0.5,0.2,0.4,0.4
2689,Manchester United,Liverpool,0,3,A,A Taylor,8,11,3,3,7,7,5,2,4,1,0,0,16:00,3.720000,4.125625,1.901875,1.462,2.871,2.005000,1.933333,1.9,2.2,16.3,18.6,6.7,5.9,6.9,7.7,1.2,1.4,9.3,13.0,0.5,0.5,0.3,0.3


In [35]:
# Initialize new columns
db['loss_rate_home_L10'] = None
db['loss_rate_away_L10'] = None

# Loop to calculate loss rates
loss_rate_home = []
loss_rate_away = []

for idx, row in db.iterrows():
    home_team = row['HomeTeam']
    away_team = row['AwayTeam']

    # Last 10 home games of home team
    past_home = db[(db.index < idx) & (db['HomeTeam'] == home_team)]
    recent_home = past_home.tail(10)
    home_losses = (recent_home['FTR'] == 'A').sum()
    loss_rate_home.append(home_losses / len(recent_home) if len(recent_home) > 0 else None)

    # Last 10 away games of away team
    past_away = db[(db.index < idx) & (db['AwayTeam'] == away_team)]
    recent_away = past_away.tail(10)
    away_losses = (recent_away['FTR'] == 'H').sum()
    loss_rate_away.append(away_losses / len(recent_away) if len(recent_away) > 0 else None)

# Assign values
db['loss_rate_home_L10'] = loss_rate_home
db['loss_rate_away_L10'] = loss_rate_away


In [36]:
db.tail()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,Time,odds_hw,odds_d,odds_aw,odds_over2_5,odds_under2_5,odds_home_asian,odds_away_asian,avg_goals_L10_home,avg_goals_L10_away,avg_shots_home_L10,avg_shots_away_L10,avg_shots_ont_home_L10,avg_shots_ont_away_L10,avg_corners_home_L10,avg_corners_away_L10,avg_goals_conceded_home_L10,avg_goals_conceded_away_L10,avg_fouls_committed_home_L10,avg_fouls_committed_away_L10,win_rate_home_L10,win_rate_away_L10,draw_rate_home_L10,draw_rate_away_L10,loss_rate_home_L10,loss_rate_away_L10
2685,Nottingham Forest,Wolverhampton Wanderers,1,1,D,S Hooper,16,11,5,3,15,18,7,3,3,4,0,0,15:00,2.024375,3.643125,3.720000,1.879,2.030,2.036667,1.905000,1.4,1.0,14.3,9.3,4.8,3.6,4.2,2.5,1.6,2.0,11.4,13.1,0.2,0.2,0.3,0.3,0.5,0.5
2686,West Ham,Manchester City,1,3,A,M Oliver,10,23,2,8,10,3,3,11,3,2,0,0,17:30,7.575000,5.280625,1.396875,1.518,2.690,1.936667,1.991667,1.5,2.6,14.1,15.5,4.6,7.6,4.9,7.0,2.0,0.6,11.3,6.5,0.2,0.9,0.5,0.1,0.3,0.0
2687,Chelsea,Crystal Palace,1,1,D,J Gillett,13,9,7,3,9,13,4,5,4,2,0,0,13:30,1.643125,4.344375,5.000000,1.545,2.599,1.950000,1.985000,2.7,1.0,18.6,9.5,7.8,3.8,6.6,4.4,1.4,1.9,10.5,13.2,0.7,0.2,0.1,0.3,0.2,0.5
2688,Newcastle United,Tottenham,2,1,H,R Jones,9,20,3,6,16,13,7,12,4,4,0,0,13:30,2.500000,3.950000,2.596250,1.405,3.118,1.928333,2.008333,2.7,1.5,16.3,13.5,5.8,5.3,5.9,6.6,1.5,1.9,10.2,11.4,0.5,0.2,0.4,0.4,0.1,0.4
2689,Manchester United,Liverpool,0,3,A,A Taylor,8,11,3,3,7,7,5,2,4,1,0,0,16:00,3.720000,4.125625,1.901875,1.462,2.871,2.005000,1.933333,1.9,2.2,16.3,18.6,6.7,5.9,6.9,7.7,1.2,1.4,9.3,13.0,0.5,0.5,0.3,0.3,0.2,0.2


In [39]:
# Create the feature 'home_strength_diff'
db['home_strength_diff'] = (
    (db['avg_goals_L10_home'] - db['avg_goals_conceded_home_L10']) -
    (db['avg_goals_L10_away'] - db['avg_goals_conceded_away_L10'])
)


In [40]:
db.tail()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,Time,odds_hw,odds_d,odds_aw,odds_over2_5,odds_under2_5,odds_home_asian,odds_away_asian,avg_goals_L10_home,avg_goals_L10_away,avg_shots_home_L10,avg_shots_away_L10,avg_shots_ont_home_L10,avg_shots_ont_away_L10,avg_corners_home_L10,avg_corners_away_L10,avg_goals_conceded_home_L10,avg_goals_conceded_away_L10,avg_fouls_committed_home_L10,avg_fouls_committed_away_L10,win_rate_home_L10,win_rate_away_L10,draw_rate_home_L10,draw_rate_away_L10,loss_rate_home_L10,loss_rate_away_L10,home_strength_diff
2685,Nottingham Forest,Wolverhampton Wanderers,1,1,D,S Hooper,16,11,5,3,15,18,7,3,3,4,0,0,15:00,2.024375,3.643125,3.720000,1.879,2.030,2.036667,1.905000,1.4,1.0,14.3,9.3,4.8,3.6,4.2,2.5,1.6,2.0,11.4,13.1,0.2,0.2,0.3,0.3,0.5,0.5,0.8
2686,West Ham,Manchester City,1,3,A,M Oliver,10,23,2,8,10,3,3,11,3,2,0,0,17:30,7.575000,5.280625,1.396875,1.518,2.690,1.936667,1.991667,1.5,2.6,14.1,15.5,4.6,7.6,4.9,7.0,2.0,0.6,11.3,6.5,0.2,0.9,0.5,0.1,0.3,0.0,-2.5
2687,Chelsea,Crystal Palace,1,1,D,J Gillett,13,9,7,3,9,13,4,5,4,2,0,0,13:30,1.643125,4.344375,5.000000,1.545,2.599,1.950000,1.985000,2.7,1.0,18.6,9.5,7.8,3.8,6.6,4.4,1.4,1.9,10.5,13.2,0.7,0.2,0.1,0.3,0.2,0.5,2.2
2688,Newcastle United,Tottenham,2,1,H,R Jones,9,20,3,6,16,13,7,12,4,4,0,0,13:30,2.500000,3.950000,2.596250,1.405,3.118,1.928333,2.008333,2.7,1.5,16.3,13.5,5.8,5.3,5.9,6.6,1.5,1.9,10.2,11.4,0.5,0.2,0.4,0.4,0.1,0.4,1.6
2689,Manchester United,Liverpool,0,3,A,A Taylor,8,11,3,3,7,7,5,2,4,1,0,0,16:00,3.720000,4.125625,1.901875,1.462,2.871,2.005000,1.933333,1.9,2.2,16.3,18.6,6.7,5.9,6.9,7.7,1.2,1.4,9.3,13.0,0.5,0.5,0.3,0.3,0.2,0.2,-0.1


In [41]:
# Create the features 'offensive_index_home' and 'offensive_index_away'
db['offensive_index_home'] = (
    0.4 * db['avg_goals_L10_home'] +
    0.3 * db['avg_shots_home_L10'] +
    0.3 * db['avg_shots_ont_home_L10']
)

db['offensive_index_away'] = (
    0.4 * db['avg_goals_L10_away'] +
    0.3 * db['avg_shots_away_L10'] +
    0.3 * db['avg_shots_ont_away_L10']
)


In [42]:
db.tail()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,Time,odds_hw,odds_d,odds_aw,odds_over2_5,odds_under2_5,odds_home_asian,odds_away_asian,avg_goals_L10_home,avg_goals_L10_away,avg_shots_home_L10,avg_shots_away_L10,avg_shots_ont_home_L10,avg_shots_ont_away_L10,avg_corners_home_L10,avg_corners_away_L10,avg_goals_conceded_home_L10,avg_goals_conceded_away_L10,avg_fouls_committed_home_L10,avg_fouls_committed_away_L10,win_rate_home_L10,win_rate_away_L10,draw_rate_home_L10,draw_rate_away_L10,loss_rate_home_L10,loss_rate_away_L10,home_strength_diff,offensive_index_home,offensive_index_away
2685,Nottingham Forest,Wolverhampton Wanderers,1,1,D,S Hooper,16,11,5,3,15,18,7,3,3,4,0,0,15:00,2.024375,3.643125,3.720000,1.879,2.030,2.036667,1.905000,1.4,1.0,14.3,9.3,4.8,3.6,4.2,2.5,1.6,2.0,11.4,13.1,0.2,0.2,0.3,0.3,0.5,0.5,0.8,6.29,4.27
2686,West Ham,Manchester City,1,3,A,M Oliver,10,23,2,8,10,3,3,11,3,2,0,0,17:30,7.575000,5.280625,1.396875,1.518,2.690,1.936667,1.991667,1.5,2.6,14.1,15.5,4.6,7.6,4.9,7.0,2.0,0.6,11.3,6.5,0.2,0.9,0.5,0.1,0.3,0.0,-2.5,6.21,7.97
2687,Chelsea,Crystal Palace,1,1,D,J Gillett,13,9,7,3,9,13,4,5,4,2,0,0,13:30,1.643125,4.344375,5.000000,1.545,2.599,1.950000,1.985000,2.7,1.0,18.6,9.5,7.8,3.8,6.6,4.4,1.4,1.9,10.5,13.2,0.7,0.2,0.1,0.3,0.2,0.5,2.2,9.00,4.39
2688,Newcastle United,Tottenham,2,1,H,R Jones,9,20,3,6,16,13,7,12,4,4,0,0,13:30,2.500000,3.950000,2.596250,1.405,3.118,1.928333,2.008333,2.7,1.5,16.3,13.5,5.8,5.3,5.9,6.6,1.5,1.9,10.2,11.4,0.5,0.2,0.4,0.4,0.1,0.4,1.6,7.71,6.24
2689,Manchester United,Liverpool,0,3,A,A Taylor,8,11,3,3,7,7,5,2,4,1,0,0,16:00,3.720000,4.125625,1.901875,1.462,2.871,2.005000,1.933333,1.9,2.2,16.3,18.6,6.7,5.9,6.9,7.7,1.2,1.4,9.3,13.0,0.5,0.5,0.3,0.3,0.2,0.2,-0.1,7.66,8.23


In [43]:
# Create the features 'defensive_solidity_home' and 'defensive_solidity_away'
db['defensive_solidity_home'] = (
    0.5 * db['avg_goals_conceded_home_L10'] +
    0.25 * db['avg_fouls_committed_home_L10'] +
    0.25 * (1 - db['win_rate_home_L10'])  # losing often hints at weaker defense
)

db['defensive_solidity_away'] = (
    0.5 * db['avg_goals_conceded_away_L10'] +
    0.25 * db['avg_fouls_committed_away_L10'] +
    0.25 * (1 - db['win_rate_away_L10'])
)


In [44]:
db.tail()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,Time,odds_hw,odds_d,odds_aw,odds_over2_5,odds_under2_5,odds_home_asian,odds_away_asian,avg_goals_L10_home,avg_goals_L10_away,avg_shots_home_L10,avg_shots_away_L10,avg_shots_ont_home_L10,avg_shots_ont_away_L10,avg_corners_home_L10,avg_corners_away_L10,avg_goals_conceded_home_L10,avg_goals_conceded_away_L10,avg_fouls_committed_home_L10,avg_fouls_committed_away_L10,win_rate_home_L10,win_rate_away_L10,draw_rate_home_L10,draw_rate_away_L10,loss_rate_home_L10,loss_rate_away_L10,home_strength_diff,offensive_index_home,offensive_index_away,defensive_solidity_home,defensive_solidity_away
2685,Nottingham Forest,Wolverhampton Wanderers,1,1,D,S Hooper,16,11,5,3,15,18,7,3,3,4,0,0,15:00,2.024375,3.643125,3.720000,1.879,2.030,2.036667,1.905000,1.4,1.0,14.3,9.3,4.8,3.6,4.2,2.5,1.6,2.0,11.4,13.1,0.2,0.2,0.3,0.3,0.5,0.5,0.8,6.29,4.27,3.850,4.475
2686,West Ham,Manchester City,1,3,A,M Oliver,10,23,2,8,10,3,3,11,3,2,0,0,17:30,7.575000,5.280625,1.396875,1.518,2.690,1.936667,1.991667,1.5,2.6,14.1,15.5,4.6,7.6,4.9,7.0,2.0,0.6,11.3,6.5,0.2,0.9,0.5,0.1,0.3,0.0,-2.5,6.21,7.97,4.025,1.950
2687,Chelsea,Crystal Palace,1,1,D,J Gillett,13,9,7,3,9,13,4,5,4,2,0,0,13:30,1.643125,4.344375,5.000000,1.545,2.599,1.950000,1.985000,2.7,1.0,18.6,9.5,7.8,3.8,6.6,4.4,1.4,1.9,10.5,13.2,0.7,0.2,0.1,0.3,0.2,0.5,2.2,9.00,4.39,3.400,4.450
2688,Newcastle United,Tottenham,2,1,H,R Jones,9,20,3,6,16,13,7,12,4,4,0,0,13:30,2.500000,3.950000,2.596250,1.405,3.118,1.928333,2.008333,2.7,1.5,16.3,13.5,5.8,5.3,5.9,6.6,1.5,1.9,10.2,11.4,0.5,0.2,0.4,0.4,0.1,0.4,1.6,7.71,6.24,3.425,4.000
2689,Manchester United,Liverpool,0,3,A,A Taylor,8,11,3,3,7,7,5,2,4,1,0,0,16:00,3.720000,4.125625,1.901875,1.462,2.871,2.005000,1.933333,1.9,2.2,16.3,18.6,6.7,5.9,6.9,7.7,1.2,1.4,9.3,13.0,0.5,0.5,0.3,0.3,0.2,0.2,-0.1,7.66,8.23,3.050,4.075


In [45]:
# Create the features 'avg_home_yellow_L10' and 'avg_away_yellow_L10'
db['avg_home_yellow_L10'] = None
db['avg_away_yellow_L10'] = None

home_yellow = []
away_yellow = []

for idx, row in db.iterrows():
    home_team = row['HomeTeam']
    away_team = row['AwayTeam']

    past_home = db[(db.index < idx) & (db['HomeTeam'] == home_team)]
    avg_home_yellow = past_home['HY'].tail(10).mean()
    home_yellow.append(avg_home_yellow)

    past_away = db[(db.index < idx) & (db['AwayTeam'] == away_team)]
    avg_away_yellow = past_away['AY'].tail(10).mean()
    away_yellow.append(avg_away_yellow)

db['avg_home_yellow_L10'] = home_yellow
db['avg_away_yellow_L10'] = away_yellow


In [46]:
# Create the features 'avg_home_red_L10' and 'avg_away_red_L10'
db['avg_home_red_L10'] = None
db['avg_away_red_L10'] = None

home_red = []
away_red = []

for idx, row in db.iterrows():
    home_team = row['HomeTeam']
    away_team = row['AwayTeam']

    past_home = db[(db.index < idx) & (db['HomeTeam'] == home_team)]
    avg_home_red = past_home['HR'].tail(10).mean()
    home_red.append(avg_home_red)

    past_away = db[(db.index < idx) & (db['AwayTeam'] == away_team)]
    avg_away_red = past_away['AR'].tail(10).mean()
    away_red.append(avg_away_red)

db['avg_home_red_L10'] = home_red
db['avg_away_red_L10'] = away_red


In [47]:
db.tail()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,Time,odds_hw,odds_d,odds_aw,odds_over2_5,odds_under2_5,odds_home_asian,odds_away_asian,avg_goals_L10_home,avg_goals_L10_away,avg_shots_home_L10,avg_shots_away_L10,avg_shots_ont_home_L10,avg_shots_ont_away_L10,avg_corners_home_L10,avg_corners_away_L10,avg_goals_conceded_home_L10,avg_goals_conceded_away_L10,avg_fouls_committed_home_L10,avg_fouls_committed_away_L10,win_rate_home_L10,win_rate_away_L10,draw_rate_home_L10,draw_rate_away_L10,loss_rate_home_L10,loss_rate_away_L10,home_strength_diff,offensive_index_home,offensive_index_away,defensive_solidity_home,defensive_solidity_away,avg_home_yellow_L10,avg_away_yellow_L10,avg_home_red_L10,avg_away_red_L10
2685,Nottingham Forest,Wolverhampton Wanderers,1,1,D,S Hooper,16,11,5,3,15,18,7,3,3,4,0,0,15:00,2.024375,3.643125,3.720000,1.879,2.030,2.036667,1.905000,1.4,1.0,14.3,9.3,4.8,3.6,4.2,2.5,1.6,2.0,11.4,13.1,0.2,0.2,0.3,0.3,0.5,0.5,0.8,6.29,4.27,3.850,4.475,1.3,2.4,0.0,0.1
2686,West Ham,Manchester City,1,3,A,M Oliver,10,23,2,8,10,3,3,11,3,2,0,0,17:30,7.575000,5.280625,1.396875,1.518,2.690,1.936667,1.991667,1.5,2.6,14.1,15.5,4.6,7.6,4.9,7.0,2.0,0.6,11.3,6.5,0.2,0.9,0.5,0.1,0.3,0.0,-2.5,6.21,7.97,4.025,1.950,1.5,1.2,0.0,0.0
2687,Chelsea,Crystal Palace,1,1,D,J Gillett,13,9,7,3,9,13,4,5,4,2,0,0,13:30,1.643125,4.344375,5.000000,1.545,2.599,1.950000,1.985000,2.7,1.0,18.6,9.5,7.8,3.8,6.6,4.4,1.4,1.9,10.5,13.2,0.7,0.2,0.1,0.3,0.2,0.5,2.2,9.00,4.39,3.400,4.450,2.1,2.0,0.0,0.1
2688,Newcastle United,Tottenham,2,1,H,R Jones,9,20,3,6,16,13,7,12,4,4,0,0,13:30,2.500000,3.950000,2.596250,1.405,3.118,1.928333,2.008333,2.7,1.5,16.3,13.5,5.8,5.3,5.9,6.6,1.5,1.9,10.2,11.4,0.5,0.2,0.4,0.4,0.1,0.4,1.6,7.71,6.24,3.425,4.000,1.0,2.0,0.2,0.0
2689,Manchester United,Liverpool,0,3,A,A Taylor,8,11,3,3,7,7,5,2,4,1,0,0,16:00,3.720000,4.125625,1.901875,1.462,2.871,2.005000,1.933333,1.9,2.2,16.3,18.6,6.7,5.9,6.9,7.7,1.2,1.4,9.3,13.0,0.5,0.5,0.3,0.3,0.2,0.2,-0.1,7.66,8.23,3.050,4.075,1.6,1.6,0.0,0.1


Now we can drop the in game statistic columns:

In [48]:
db.drop(columns=[
    'FTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR'
], inplace=True)


In [49]:
db.tail()

,HomeTeam,AwayTeam,FTHG,FTAG,Referee,Time,odds_hw,odds_d,odds_aw,odds_over2_5,odds_under2_5,odds_home_asian,odds_away_asian,avg_goals_L10_home,avg_goals_L10_away,avg_shots_home_L10,avg_shots_away_L10,avg_shots_ont_home_L10,avg_shots_ont_away_L10,avg_corners_home_L10,avg_corners_away_L10,avg_goals_conceded_home_L10,avg_goals_conceded_away_L10,avg_fouls_committed_home_L10,avg_fouls_committed_away_L10,win_rate_home_L10,win_rate_away_L10,draw_rate_home_L10,draw_rate_away_L10,loss_rate_home_L10,loss_rate_away_L10,home_strength_diff,offensive_index_home,offensive_index_away,defensive_solidity_home,defensive_solidity_away,avg_home_yellow_L10,avg_away_yellow_L10,avg_home_red_L10,avg_away_red_L10
2685,Nottingham Forest,Wolverhampton Wanderers,1,1,S Hooper,15:00,2.024375,3.643125,3.720000,1.879,2.030,2.036667,1.905000,1.4,1.0,14.3,9.3,4.8,3.6,4.2,2.5,1.6,2.0,11.4,13.1,0.2,0.2,0.3,0.3,0.5,0.5,0.8,6.29,4.27,3.850,4.475,1.3,2.4,0.0,0.1
2686,West Ham,Manchester City,1,3,M Oliver,17:30,7.575000,5.280625,1.396875,1.518,2.690,1.936667,1.991667,1.5,2.6,14.1,15.5,4.6,7.6,4.9,7.0,2.0,0.6,11.3,6.5,0.2,0.9,0.5,0.1,0.3,0.0,-2.5,6.21,7.97,4.025,1.950,1.5,1.2,0.0,0.0
2687,Chelsea,Crystal Palace,1,1,J Gillett,13:30,1.643125,4.344375,5.000000,1.545,2.599,1.950000,1.985000,2.7,1.0,18.6,9.5,7.8,3.8,6.6,4.4,1.4,1.9,10.5,13.2,0.7,0.2,0.1,0.3,0.2,0.5,2.2,9.00,4.39,3.400,4.450,2.1,2.0,0.0,0.1
2688,Newcastle United,Tottenham,2,1,R Jones,13:30,2.500000,3.950000,2.596250,1.405,3.118,1.928333,2.008333,2.7,1.5,16.3,13.5,5.8,5.3,5.9,6.6,1.5,1.9,10.2,11.4,0.5,0.2,0.4,0.4,0.1,0.4,1.6,7.71,6.24,3.425,4.000,1.0,2.0,0.2,0.0
2689,Manchester United,Liverpool,0,3,A Taylor,16:00,3.720000,4.125625,1.901875,1.462,2.871,2.005000,1.933333,1.9,2.2,16.3,18.6,6.7,5.9,6.9,7.7,1.2,1.4,9.3,13.0,0.5,0.5,0.3,0.3,0.2,0.2,-0.1,7.66,8.23,3.050,4.075,1.6,1.6,0.0,0.1


In [50]:
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2690 entries, 0 to 2689
Data columns (total 40 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   HomeTeam                      2690 non-null   object 
 1   AwayTeam                      2690 non-null   object 
 2   FTHG                          2690 non-null   int64  
 3   FTAG                          2690 non-null   int64  
 4   Referee                       2690 non-null   object 
 5   Time                          1930 non-null   object 
 6   odds_hw                       2690 non-null   float64
 7   odds_d                        2690 non-null   float64
 8   odds_aw                       2690 non-null   float64
 9   odds_over2_5                  2690 non-null   float64
 10  odds_under2_5                 2690 non-null   float64
 11  odds_home_asian               1930 non-null   float64
 12  odds_away_asian               1930 non-null   float64
 13  avg

In [51]:
# Initialize new columns
db['h2h_avg_home_goals_last_5'] = None
db['h2h_avg_away_goals_last_5'] = None

# Loop through each match to compute head-to-head averages
for idx, row in db.iterrows():
    home_team = row['HomeTeam']
    away_team = row['AwayTeam']

    # Get last 5 matches between these two teams (any order), before current match
    past_h2h = db[
        (db.index < idx) & (
            ((db['HomeTeam'] == home_team) & (db['AwayTeam'] == away_team)) |
            ((db['HomeTeam'] == away_team) & (db['AwayTeam'] == home_team))
        )
    ].tail(5)

    home_goals = []
    away_goals = []

    for _, match in past_h2h.iterrows():
        if match['HomeTeam'] == home_team:
            home_goals.append(match['FTHG'])
            away_goals.append(match['FTAG'])
        else:
            home_goals.append(match['FTAG'])
            away_goals.append(match['FTHG'])

    db.at[idx, 'h2h_avg_home_goals_last_5'] = sum(home_goals) / len(home_goals) if home_goals else None
    db.at[idx, 'h2h_avg_away_goals_last_5'] = sum(away_goals) / len(away_goals) if away_goals else None


In [52]:
db.tail()

,HomeTeam,AwayTeam,FTHG,FTAG,Referee,Time,odds_hw,odds_d,odds_aw,odds_over2_5,odds_under2_5,odds_home_asian,odds_away_asian,avg_goals_L10_home,avg_goals_L10_away,avg_shots_home_L10,avg_shots_away_L10,avg_shots_ont_home_L10,avg_shots_ont_away_L10,avg_corners_home_L10,avg_corners_away_L10,avg_goals_conceded_home_L10,avg_goals_conceded_away_L10,avg_fouls_committed_home_L10,avg_fouls_committed_away_L10,win_rate_home_L10,win_rate_away_L10,draw_rate_home_L10,draw_rate_away_L10,loss_rate_home_L10,loss_rate_away_L10,home_strength_diff,offensive_index_home,offensive_index_away,defensive_solidity_home,defensive_solidity_away,avg_home_yellow_L10,avg_away_yellow_L10,avg_home_red_L10,avg_away_red_L10,h2h_avg_home_goals_last_5,h2h_avg_away_goals_last_5
2685,Nottingham Forest,Wolverhampton Wanderers,1,1,S Hooper,15:00,2.024375,3.643125,3.720000,1.879,2.030,2.036667,1.905000,1.4,1.0,14.3,9.3,4.8,3.6,4.2,2.5,1.6,2.0,11.4,13.1,0.2,0.2,0.3,0.3,0.5,0.5,0.8,6.29,4.27,3.850,4.475,1.3,2.4,0.0,0.1,1.0,1.25
2686,West Ham,Manchester City,1,3,M Oliver,17:30,7.575000,5.280625,1.396875,1.518,2.690,1.936667,1.991667,1.5,2.6,14.1,15.5,4.6,7.6,4.9,7.0,2.0,0.6,11.3,6.5,0.2,0.9,0.5,0.1,0.3,0.0,-2.5,6.21,7.97,4.025,1.950,1.5,1.2,0.0,0.0,0.8,2.6
2687,Chelsea,Crystal Palace,1,1,J Gillett,13:30,1.643125,4.344375,5.000000,1.545,2.599,1.950000,1.985000,2.7,1.0,18.6,9.5,7.8,3.8,6.6,4.4,1.4,1.9,10.5,13.2,0.7,0.2,0.1,0.3,0.2,0.5,2.2,9.00,4.39,3.400,4.450,2.1,2.0,0.0,0.1,1.8,0.6
2688,Newcastle United,Tottenham,2,1,R Jones,13:30,2.500000,3.950000,2.596250,1.405,3.118,1.928333,2.008333,2.7,1.5,16.3,13.5,5.8,5.3,5.9,6.6,1.5,1.9,10.2,11.4,0.5,0.2,0.4,0.4,0.1,0.4,1.6,7.71,6.24,3.425,4.000,1.0,2.0,0.2,0.0,2.8,2.2
2689,Manchester United,Liverpool,0,3,A Taylor,16:00,3.720000,4.125625,1.901875,1.462,2.871,2.005000,1.933333,1.9,2.2,16.3,18.6,6.7,5.9,6.9,7.7,1.2,1.4,9.3,13.0,0.5,0.5,0.3,0.3,0.2,0.2,-0.1,7.66,8.23,3.050,4.075,1.6,1.6,0.0,0.1,0.8,2.8


# Plots